In [1]:
import numpy as np
import torch

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [5]:
# Weights and Biases
# torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribuion with the mean = 0 and standrad deviation = True
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.4015, -1.1581, -1.4558],
        [-0.7886, -0.3937,  0.8726]], requires_grad=True)
tensor([ 0.6739, -0.6075], requires_grad=True)


In [14]:
# Creating Model
# @ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.
def model(x):
    return x @ w.t() + b

In [8]:
# Generating the predictions
predict = model(inputs)
print(predict)

tensor([[-168.8257,  -47.0285],
        [-230.9441,  -51.1657],
        [-273.8743,  -71.3566],
        [-143.9404,  -65.6840],
        [-240.1108,  -31.7311]], grad_fn=<AddBackward0>)


In [9]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [12]:
# MSE loss
# numel() = Number of Elements
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff*diff)/diff.numel() # Note: diff^2 cannot be used to square the elements, because Pytorch captures the derivatives behind the arithmetic operatons.

In [15]:
# Compute Loss
loss = mse(predict, targets)
print(loss)

tensor(55934.2734, grad_fn=<DivBackward0>)


In [16]:
# Lets Compute Gradients
loss.backward()

In [17]:
# Gradients fro weights
print(w)
print(w.grad)

tensor([[-0.4015, -1.1581, -1.4558],
        [-0.7886, -0.3937,  0.8726]], requires_grad=True)
tensor([[-23915.9648, -27046.7246, -16515.2363],
        [-12208.6797, -13500.1758,  -8194.7998]])


In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [19]:
predict = model(inputs)
print(predict)

tensor([[-168.8257,  -47.0285],
        [-230.9441,  -51.1657],
        [-273.8743,  -71.3566],
        [-143.9404,  -65.6840],
        [-240.1108,  -31.7311]], grad_fn=<AddBackward0>)


In [20]:
loss = mse(predict, targets)
print(loss)

tensor(55934.2734, grad_fn=<DivBackward0>)


In [21]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-23915.9648, -27046.7246, -16515.2363],
        [-12208.6797, -13500.1758,  -8194.7998]])
tensor([-287.7390, -145.3932])


In [24]:
# Adjust weights and reset gradients
# "1e-5" is the small quantity of adjustment
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [25]:
print(w)

tensor([[-0.1623, -0.8876, -1.2907],
        [-0.6665, -0.2587,  0.9545]], requires_grad=True)


In [26]:
print(b)

tensor([ 0.6825, -0.6031], requires_grad=True)


In [27]:
# Lets Calculate the loss
predict = model(inputs)
loss = mse(predict, targets)
print(loss)

tensor(37951.9648, grad_fn=<DivBackward0>)


In [29]:
# Train for mltiple epochs
for i in range(100):
    predicts = model(inputs)
    loss = mse(predicts, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [31]:
# Compute the loss
predicts = model(inputs)
loss = mse(predicts, targets)
print(loss)

tensor(114.4481, grad_fn=<DivBackward0>)


In [32]:
print(predicts)

tensor([[ 61.3545,  70.7761],
        [ 77.7161, 104.2237],
        [122.1737, 124.1239],
        [ 44.0212,  40.6378],
        [ 80.8573, 123.0878]], grad_fn=<AddBackward0>)


In [33]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
